In [2]:
# load the modules and functions

import sys
sys.path.insert(0, '../functions/')
import plot_params

import xarray as xr
import pandas as pd

import numpy as np
import cmocean.cm as cmo
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# read in the slocum data

sl = xr.open_dataset('../../../data/slocum/slocum_processed_L2.nc').load()

In [4]:
# calculate conservative temperature and absolute salinity

import gsw

# Convert salinity from SP to SA and temperature from T to CT
sa = gsw.SA_from_SP(sl['salinity'].data, sl['pressure'].data, sl['longitude'].data, sl['latitude'].data)

ct = gsw.CT_from_t(sa, sl['temperature'].data, sl['pressure'].data)

sl['con_temp'] = (('time'), ct)
sl['abs_salt'] = (('time'), sa)

# Calculate seawater density using gsw
sl['density'] = gsw.rho(sl['abs_salt'].data, sl['con_temp'].data, sl['pressure'].data / 1013.25)  

In [5]:
# grid the slocum data to 1m depth intervals

from scipy.interpolate import griddata

new_z = np.arange(0, 1001, 1)
time = np.array([], dtype='datetime64[ns]')

lat = np.array([])
lon = np.array([])

temp = np.ndarray([np.unique(sl.dive.data).size, new_z.size])
salt = np.ndarray([np.unique(sl.dive.data).size, new_z.size])
dens = np.ndarray([np.unique(sl.dive.data).size, new_z.size])

for i, d in enumerate(np.unique(sl.dive.data)):

    dive = sl.isel(time=sl.dive==d)

    temp[i] = griddata(dive.depth.data, dive.con_temp.data, new_z)
    salt[i] = griddata(dive.depth.data, dive.abs_salt.data, new_z)
    dens[i] = griddata(sorted(dive.depth.data), dive.density.data, new_z)
  
    time = np.append(time, dive.time.min())
    lon  = np.append(lon, np.nanmedian(dive.longitude.data))
    lat  = np.append(lat, np.nanmedian(dive.latitude.data))  

In [6]:
# CREATE AN XARRAY DATASET FOR THE SLOCUM DATA

# xarray dataset for slocum gridded data

sl_grid = xr.Dataset(
    {
        'temperature': (['depth', 'time'], temp.T),
        'salinity': (['depth', 'time'], salt.T),
        'density': (['depth', 'time'], dens.T),
        'latitude': (['time'], lat),
        'longitude': (['time'], lon),        
    },
    coords={
        'time': time,
        'depth': new_z
    }
)

# Optionally add attributes
sl_grid['temperature'].attrs['units'] = 'deg Celsius'
sl_grid['salinity'].attrs['units'] = 'g/kg'
sl_grid['density'].attrs['units'] = 'kg/m^3'
sl_grid['temperature'].attrs['long_name'] = 'Converative Temperature'
sl_grid['salinity'].attrs['long_name'] = 'Absolute Salinity'
sl_grid['density'].attrs['long_name'] = 'Seawater Density'

In [7]:
# CALCULATE STRATIFICATION FROM THE GLIDER

g = 9.81

N2_T = np.ndarray([np.unique(sl.dive.data).size, new_z.size])
N2_S = np.ndarray([np.unique(sl.dive.data).size, new_z.size])

for i in range(sl_grid.time.size):

    dive = sl_grid.isel(time=i)

    alpha = gsw.alpha(dive['salinity'].data, dive['temperature'].data, dive['depth'].data)[1:]
    beta = gsw.beta(dive['salinity'].data, dive['temperature'].data, dive['depth'].data)[1:]

    N2_T[i] = np.append(-g*alpha*(dive['temperature'].diff(dim='depth')), np.nan)
    N2_S[i] = np.append( g*beta*(dive['salinity'].diff(dim='depth')), np.nan)

sl_grid['N2_T'] = (('depth', 'time'), N2_T.T)
sl_grid['N2_S'] = (('depth', 'time'), N2_S.T)

sl_grid['N2'] = (('depth', 'time'), N2_S.T + N2_T.T)

In [8]:
# calculate the mixed layer depth

from calc_mld import calc_mld

mld = calc_mld(sl_grid.temperature.T.data, sl_grid.depth.data, den_lim=0.2)

sl_grid['mld_t'] = (('time'), mld)

In [9]:
# read the wave glider and the mixing data sets

ep = xr.open_dataset('/Users/xduplm/Google Drive/My Drive/data/duplessis-storms-paper/slocum/slocum_eps_ds_processed_era5_23Sep2022.nc')
wg = xr.open_dataset('/Users/xduplm/Google Drive/My Drive/data/duplessis-storms-paper/waveglider/WG_era5_1h_processed_28Aug2022.nc')

xld = pd.read_csv('/Users/xduplm/Google Drive/My Drive/data/duplessis-storms-paper/slocum/mixing_layer_xld.csv')

xld = xr.Dataset(data_vars=dict(
                                   xld = (['time'], xld.xld.values)),                   
                    coords=dict(
                                time = (['time'], xld.Date.values.astype('datetime64[ns]'))),                   
                    attrs=dict(
                               description="Processed Slocum glider XLD data."))

xld = xld.isel(time=xld.xld>0)


In [10]:
# determine the day of year variable for hte various datasets

import matplotlib.dates as mdates

day0=mdates.date2num(np.datetime64('2019-01-01 00:00:00'))

ep['DOY']=(('time'), mdates.date2num(ep.TIME)-day0)
xld['DOY']=(('time'), mdates.date2num(xld.time)-day0)
sl_grid['DOY']=(('time'), mdates.date2num(sl_grid.time)-day0)

In [11]:
# cut the slocum data to meet the data length of the mixing layer depth

sl_grid = sl_grid.sel(time=slice(xld.time[0].data, xld.time[-1].data))


In [12]:
sl_grid.to_netcdf('../../../data/slocum/sl_gridded.nc')

In [13]:
xld

<xarray.Dataset>
Dimensions:  (time: 786)
Coordinates:
  * time     (time) datetime64[ns] 2018-12-19T21:39:40 ... 2019-03-08T09:37:55
Data variables:
    xld      (time) float64 43.35 44.75 45.0 45.0 ... 109.8 130.7 91.72 41.03
    DOY      (time) float64 -12.1 -12.01 -11.92 -11.83 ... 66.08 66.18 66.3 66.4
Attributes:
    description:  Processed Slocum glider XLD data.